In [42]:
import cv2
print (cv2.__version__)
#assert cv2.__version__[0] == '3', 'The fisheye module requires opencv version >= 3.0.0'
import numpy as np
import os
import glob
import matplotlib.pyplot as plt

#CHECKERBOARD = (21,14)
CHECKERBOARD = (21,14)

subpix_criteria = (cv2.TERM_CRITERIA_EPS+cv2.TERM_CRITERIA_MAX_ITER, 30, 0.1)
calibration_flags = cv2.fisheye.CALIB_RECOMPUTE_EXTRINSIC+cv2.fisheye.CALIB_CHECK_COND+cv2.fisheye.CALIB_FIX_SKEW

objp = np.zeros((1, CHECKERBOARD[0]*CHECKERBOARD[1], 3), np.float32)

objp[0,:,:2] = np.mgrid[0:CHECKERBOARD[0], 0:CHECKERBOARD[1]].T.reshape(-1, 2)

_img_shape = None
objpoints = [] # 3d point in real world space
imgpoints = [] # 2d points in image plane.images = glob.glob('*.jpg')for fname in images:

##############################

fnames = glob.glob('/media/cat/1TB/dan/*.jpg')
for fname in fnames:
    img = cv2.imread(fname)
    print (fname, img.shape)
    if _img_shape == None:
        _img_shape = img.shape[:2]
    else:
        assert _img_shape == img.shape[:2], "All images must share the same size."    

    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    #gray = img[:,:,2]

    corners = cv2.goodFeaturesToTrack(gray,1000,0.0001,100)
    print ("goodferaures to tracK: ", corners.shape)
    
    ret = True

    # Find the chess board corners
    ret, corners = cv2.findChessboardCorners(gray, 
                                             CHECKERBOARD, 
                                             cv2.CALIB_CB_ADAPTIVE_THRESH+cv2.CALIB_CB_FAST_CHECK+cv2.CALIB_CB_NORMALIZE_IMAGE)

    print (ret, "findChessboardCorners: ", corners)

#     #corners = corners.squeeze()
#     corners = np.ascontiguousarray(corners)
#     print (ret, corners.shape)

    # If found, add object points, image points (after refining them)
    if ret == True:
        objpoints.append(objp)
        cv2.cornerSubPix(gray,corners,(3,3),(-1,-1),subpix_criteria)
        imgpoints.append(corners)
        
    break
        
N_OK = 1

K = np.zeros((3, 3))
D = np.zeros((4, 1))
print (gray.shape[::-1])
print (corners.shape)
rvecs = [np.zeros((1, 1, 3), dtype=np.float64) for i in range(N_OK)]
tvecs = [np.zeros((1, 1, 3), dtype=np.float64) for i in range(N_OK)]
rms, _, _, _, _ = \
    cv2.fisheye.calibrate(
        objpoints,
        imgpoints,
        gray.shape[::-1],
        K,
        D,
        rvecs,
        tvecs,
        calibration_flags,
        (cv2.TERM_CRITERIA_EPS+cv2.TERM_CRITERIA_MAX_ITER, 30, 1e-6)
    )
print("Found " + str(N_OK) + " valid images for calibration")
print("DIM=" + str(_img_shape[::-1]))
print("K=np.array(" + str(K.tolist()) + ")")
print("D=np.array(" + str(D.tolist()) + ")")

4.5.1
/media/cat/1TB/dan/checkerboard2.jpg (1024, 1280, 3)
goodferaures to tracK:  (57, 1, 2)
False findChessboardCorners:  None
(1280, 1024)


AttributeError: 'NoneType' object has no attribute 'shape'

In [47]:
import numpy as np
import cv2
import glob
# termination criteria
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.1)
# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
objp = np.zeros((6*7,3), np.float32)
objp[:,:2] = np.mgrid[0:7,0:6].T.reshape(-1,2)
# Arrays to store object points and image points from all the images.
objpoints = [] # 3d point in real world space
imgpoints = [] # 2d points in image plane.
images = glob.glob('/media/cat/1TB/dan/*.jpg')
for fname in images:
    img = cv2.imread(fname)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # Find the chess board corners
    #ret, corners = cv2.findChessboardCorners(gray, (9,6), None)
    #gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    #gray = img[:,:,2]
    
    ret = True
    useHarrisDetector=True
    corners = cv2.goodFeaturesToTrack(gray,1000,0.0001,100).astype('float32')
    #corners = np.int0(corners)
    
    print (ret, corners.shape)
    # If found, add object points, image points (after refining them)
    if ret == True:
        objpoints.append(objp)
        corners2=cv2.cornerSubPix(gray,corners, (11,11), (-1,-1), criteria)
        imgpoints.append(corners)
        # Draw and display the corners
        cv2.drawChessboardCorners(img, (7,6), corners2, ret)
        cv2.imshow('img', img)
        cv2.waitKey(2000)
cv2.destroyAllWindows()

# aturesToTrack 	( 	InputArray  	image,
# 		OutputArray  	corners,
# 		int  	maxCorners,
# 		double  	qualityLevel,
# 		double  	minDistance,
# 		InputArray  	mask = noArray(),
# 		int  	blockSize = 3,
# 		bool  	useHarrisDetector = false,
# 		double  	k = 0.04 
# 	) 		



True (51, 1, 2)
True (48, 1, 2)


In [59]:
img = cv2.imread(fname)
img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
print (img)
a, b = cv2.findChessboardCorners(img, (14, 15))

print (a, b)

[[255 255 255 ... 255 255 255]
 [255 255 255 ... 255 255 255]
 [255 255 255 ... 255 255 255]
 ...
 [255 255 255 ... 255 255 255]
 [255 255 255 ... 255 255 255]
 [255 255 255 ... 255 255 255]]
False None


In [48]:

# calculate K & D
N_imm = 1 # number of calibration images
K = np.zeros((3, 3))
D = np.zeros((4, 1))
rvecs = [np.zeros((1, 1, 3), dtype=np.float64) for i in range(N_imm)]
tvecs = [np.zeros((1, 1, 3), dtype=np.float64) for i in range(N_imm)]
retval, K, D, rvecs, tvecs = cv2.fisheye.calibrate(
    objpoints,
    imgpoints,
    gray.shape[::-1],
    K,
    D,
    rvecs,
    tvecs,
    calibration_flags,
    (cv2.TERM_CRITERIA_EPS+cv2.TERM_CRITERIA_MAX_ITER, 30, 1e-6)
)

error: OpenCV(4.5.1) /tmp/pip-req-build-7m_g9lbm/opencv/modules/calib3d/src/fisheye.cpp:742: error: (-215:Assertion failed) objectPoints.type() == CV_32FC3 || objectPoints.type() == CV_64FC3 in function 'calibrate'


In [28]:
# Checkboard dimensions
CHECKERBOARD = (6,9)
subpix_criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.1)
calibration_flags = cv2.fisheye.CALIB_RECOMPUTE_EXTRINSIC + cv2.fisheye.CALIB_CHECK_COND + cv2.fisheye.CALIB_FIX_SKEW
objp = np.zeros((1, CHECKERBOARD[0]*CHECKERBOARD[1], 3), np.float32)
objp[0,:,:2] = np.mgrid[0:CHECKERBOARD[0], 0:CHECKERBOARD[1]].T.reshape(-1, 2)

objpoints = [] # 3d point in real world space
imgpoints = [] # 2d points in image plane.

### read images and for each image:
fname = '/media/cat/1TB/dan/checkerboard1.jpg'
img = cv2.imread(fname)
img_shape = img.shape[:2]

gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
# Find the chess board corners
ret, corners = cv2.findChessboardCorners(gray, CHECKERBOARD, cv2.CALIB_CB_ADAPTIVE_THRESH+cv2.CALIB_CB_FAST_CHECK+cv2.CALIB_CB_NORMALIZE_IMAGE)
# If found, add object points, image points (after refining them)
if ret == True:
    objpoints.append(objp)
    cv2.cornerSubPix(gray,corners,(3,3),(-1,-1),subpix_criteria)
    imgpoints.append(corners)
###

# calculate K & D
N_imm = 1 # number of calibration images
K = np.zeros((3, 3))
D = np.zeros((4, 1))
rvecs = [np.zeros((1, 1, 3), dtype=np.float64) for i in range(N_imm)]
tvecs = [np.zeros((1, 1, 3), dtype=np.float64) for i in range(N_imm)]
retval, K, D, rvecs, tvecs = cv2.fisheye.calibrate(
    objpoints,
    imgpoints,
    gray.shape[::-1],
    K,
    D,
    rvecs,
    tvecs,
    calibration_flags,
    (cv2.TERM_CRITERIA_EPS+cv2.TERM_CRITERIA_MAX_ITER, 30, 1e-6)
)

error: OpenCV(4.5.1) /tmp/pip-req-build-7m_g9lbm/opencv/modules/calib3d/src/fisheye.cpp:741: error: (-215:Assertion failed) !objectPoints.empty() && !imagePoints.empty() && objectPoints.total() == imagePoints.total() in function 'calibrate'


In [141]:
import numpy as np

#
k = np.array([[ 512 ,   0.    ,      640],
             [   0.      ,    640  , 640],
             [   0.     ,       0.    ,        1.        ]])

d = np.array([[ 0.16],
             [ 0.0112638],
             [ 0.01004722],
             [-0.00593285]])

d = np.array([[ 0.056],
             [ 0.06],
             [ 0.05],
             [-0.00593285]])


In [144]:
from tqdm import trange, tqdm
nk = k.copy()
nk[0,0]=k[0,0]/1.5
nk[1,1]=k[1,1]/1.5
# Just by scaling the matrix coefficients!

map1, map2 = cv2.fisheye.initUndistortRectifyMap(k, d, np.eye(3), nk, 
                                                 (1280,1024), cv2.CV_16SC2)  # Pass k in 1st parameter, nk in 4th parameter

fname = '/home/cat/Downloads/cohort2/20560.jpg'
for p in trange(100):
    img = cv2.imread(fname)
    #print (img)
    nemImg = cv2.remap(img, 
                       map1, 
                       map2, 
                       interpolation=cv2.INTER_LINEAR, 
                       borderMode=cv2.BORDER_CONSTANT)

    cv2.imwrite(fname[:-4]+"_fixed.jpg", nemImg)

100%|██████████| 100/100 [00:03<00:00, 31.86it/s]


In [97]:
# Checkboard dimensions
CHECKERBOARD = (6,9)
subpix_criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.1)
calibration_flags = cv2.fisheye.CALIB_RECOMPUTE_EXTRINSIC + cv2.fisheye.CALIB_CHECK_COND + cv2.fisheye.CALIB_FIX_SKEW
objp = np.zeros((1, CHECKERBOARD[0]*CHECKERBOARD[1], 3), np.float32)
objp[0,:,:2] = np.mgrid[0:CHECKERBOARD[0], 0:CHECKERBOARD[1]].T.reshape(-1, 2)

objpoints = [] # 3d point in real world space
imgpoints = [] # 2d points in image plane.

### read images and for each image:
img = cv2.imread(fname)
img_shape = img.shape[:2]
print ("img_shape: ", img_shape)

gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

# Find the chess board corners
ret, corners = cv2.findChessboardCorners(gray, 
                                         CHECKERBOARD, 
                                         cv2.CALIB_CB_ADAPTIVE_THRESH+cv2.CALIB_CB_FAST_CHECK+cv2.CALIB_CB_NORMALIZE_IMAGE)

print (ret)
# If found, add object points, image points (after refining them)
if ret == True:
    objpoints.append(objp)
    cv2.cornerSubPix(gray,corners,(3,3),(-1,-1),subpix_criteria)
    imgpoints.append(corners)
###

# calculate K & D
N_imm = 1 # number of calibration images
K = np.zeros((3, 3))
D = np.zeros((4, 1))
rvecs = [np.zeros((1, 1, 3), dtype=np.float64) for i in range(N_imm)]
tvecs = [np.zeros((1, 1, 3), dtype=np.float64) for i in range(N_imm)]
retval, K, D, rvecs, tvecs = cv2.fisheye.calibrate(
    objpoints,
    imgpoints,
    gray.shape[::-1],
    K,
    D,
    rvecs,
    tvecs,
    calibration_flags,
    (cv2.TERM_CRITERIA_EPS+cv2.TERM_CRITERIA_MAX_ITER, 30, 1e-6))





img_shape:  (1024, 1280)
False


error: OpenCV(4.5.1) /tmp/pip-req-build-7m_g9lbm/opencv/modules/calib3d/src/fisheye.cpp:741: error: (-215:Assertion failed) !objectPoints.empty() && !imagePoints.empty() && objectPoints.total() == imagePoints.total() in function 'calibrate'


In [ ]:

if __name__ == '__main__':
    main(sys.argv)
    

In [2]:

dtype = 'linear'
format = 'fullframe'
fov = 190
pfov = 140

img = "/home/cat/Downloads/cohort2/11988.jpg"
img_out = f"/home/cat/Downloads/cohort2/11988{dtype}_{format}_{pfov}_{fov}.jpg"

obj = Defisheye(img, dtype=dtype, format=format, fov=fov, pfov=pfov)
obj.convert(img_out)

AttributeError: 'NoneType' object has no attribute 'shape'

In [86]:
import lensfunpy
import cv2 # OpenCV library

cam_maker = 'NIKON CORPORATION'
cam_model = 'NIKON D3S'
lens_maker = 'Nikon'
lens_model = 'Nikkor 28mm f/1.4D AF'

db = lensfunpy.Database()
cam = db.find_cameras(cam_maker, cam_model)[0]
print (cam)
lens = db.find_lenses(cam, lens_maker, lens_model)[0]
print (lens)



Camera(Maker: Nikon Corporation; Model: Nikon D3S; Mount: Nikon F AF; Crop Factor: 1.0; Score: 0)
Lens(Maker: Nikon; Model: Nikon AF Nikkor 28mm f/1.4D; Type: RECTILINEAR; Focal: 28.0-28.0; Aperture: 1.399999976158142-None; Crop factor: 1.0; Score: 95)


In [85]:
# #print (lensfunpy.Database.find_cameras(maker=None, model=None, loose_search=False))

# db = lensfunpy.Database()
# cam = db.find_cameras('flir')
# print (cam)
# lens = db.find_lenses(cam, 'Nikon', 'Nikkor AF 20mm f/2.8D')[0]



In [6]:
import glob
from tqdm import tqdm, trange
import os
import parmap
import lensfunpy
import cv2 # OpenCV library

focal_length = 4.5
aperture = 1.8
distance = 60
root_dir = '/home/cat/Downloads/cohort2/late/jpgs/'
root_dir_fixed = '//home/cat/Downloads/cohort2/late/jpgs_fixed/'
fnames = np.sort(glob.glob(root_dir+'/*.jpg'))

# 
#
def correct(fnames
            ):

    cam_maker = 'NIKON CORPORATION'
    cam_model = 'NIKON D3S'
    lens_maker = 'Nikon'
    lens_model = 'Nikkor 28mm f/1.4D AF'

    db = lensfunpy.Database()
    cam = db.find_cameras(cam_maker, cam_model)[0]
    lens = db.find_lenses(cam, lens_maker, lens_model)[0]

    for image_path in fnames:
        im_undistorted = cv2.imread(image_path)
        for k in range(6):
            if k==0:
                im_undistorted = np.roll(im_undistorted, -40, 0)
                im_undistorted = np.roll(im_undistorted, -40, 1)

            height, width = im_undistorted.shape[0], im_undistorted.shape[1]

            mod = lensfunpy.Modifier(lens, cam.crop_factor, width, height)
            mod.initialize(focal_length, aperture, distance)

            undist_coords = mod.apply_geometry_distortion()
            im_undistorted = cv2.remap(im_undistorted, undist_coords, None, cv2.INTER_LANCZOS4)

        undistorted_image_path = root_dir_fixed + os.path.split(image_path)[1]
        cv2.imwrite(undistorted_image_path, im_undistorted)

    
if True:
    fnames_split = np.array_split(fnames, 6)
    parmap.map(correct, 
               fnames_split,
               pm_processes=6,
              pm_pbar=True)
    

print ("DONE")
        


100%|██████████| 6/6 [01:37<00:00, 16.21s/it]

DONE
